In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install lexicalrichness

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from statistics import mean
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from lexicalrichness import LexicalRichness

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install OpenNMT-py

In [ ]:
!pip install sacremoses

In [ ]:
MYCLASSFOLDER="DSS_Thesis_code"
import os
os.environ['PATH'] += ":/content/drive/MyDrive/" + MYCLASSFOLDER + "/scripts"

In [ ]:
!MYCLASSFOLDER="DSS_Thesis_code"
!mkdir /content/drive/MyDrive/$MYCLASSFOLDER/
!ls /content/drive/MyDrive/$MYCLASSFOLDER

In [ ]:
cd /content/drive/MyDrive/$MYCLASSFOLDER

In [ ]:
!wget https://ilk.uvt.nl/~shterion/scripts/bpe.sh -O scripts/bpe.sh

In [ ]:
!mkdir scripts #Make folder with 'scripts'
!wget https://ilk.uvt.nl/~shterion/scripts/train_test_dev.py -O scripts/train_test_dev.py
!wget https://ilk.uvt.nl/~shterion/scripts/bpe.sh -O scripts/bpe.sh

In [ ]:
#!wget https://ilk.uvt.nl/~shterion/scripts/tokenizer.perl -O scripts/tokenizer.perl
!wget https://ilk.uvt.nl/~shterion/scripts/detokenize.perl -O scripts/detokenize.perl
#!wget https://ilk.uvt.nl/~shterion/scripts/nonbreaking_prefixes.zip -O scripts/nonbreaking_prefixes.zip
#!unzip scripts/nonbreaking_prefixes.zip -d scripts/

In [ ]:
!wget https://ilk.uvt.nl/~shterion/data/EN-NL.zip -O EN-NL.zip
!unzip -K EN-NL.zip

In [ ]:
df = pd.read_excel(r'/content/drive/My Drive/DSS_Thesis_code/Thesis_Data_Merged.xlsx') #Loading the data

In [ ]:
print(len(df['Spoken'].unique())) #Test to see how long the aligned data set should be

In [ ]:
data = df.groupby('Spoken')['Sign'].apply(lambda group_series: group_series.tolist()).reset_index() #Allocates all the snippits to their correct counterpart in list format

In [ ]:
for index in range(len(data['Sign'])):
  data['Sign'][index] = [word for word in data['Sign'][index] if not(pd.isnull(word)) == True] #Removing all the nan's from the data using help from https://www.pythonpool.com/python-remove-nan-from-list/

In [ ]:
for index in range(len(data['Sign'])):
  data['Sign'][index] = ' '.join([str(i) for i in data['Sign'][index]]) #Transforms the lists into strings 

In [ ]:
for index in range(len(data['Spoken'])):
  data['Spoken'][index] = data['Spoken'][index].replace('ß', 'ss') #Replacing all the 'ß' with 'ss' in the Spoken column
  


In [ ]:
for index in range(len(data['Sign'])):
  data['Sign'][index] = data['Sign'][index].replace('ß', 'ss') #Replacing all the 'ß' with 'ss' in the Sign column (just to be sure)

In [ ]:
nltk.download('stopwords') #Import all stopwords

In [ ]:
sw = stopwords.words('german')#Import all German stopwords

In [ ]:
def compression(data):
    word = data.split()
    return ' '.join([z for z in word if not z in sw])


data['Compressed']  = data['Spoken'].apply(compression) #apply compression function to data  https://stackoverflow.com/questions/57374542/i-want-to-remove-all-the-german-stop-words-from-data-set


In [ ]:
data

In [ ]:
source = data['Sign']
target = data['Spoken']
target_compressed = data['Compressed']

In [ ]:
comp = []
nor = []
for i in range(len(target_compressed)):
  comp.append(len(target_compressed[i]))
for z in range(len(target)):
  nor.append(len(target[z]))

In [ ]:
ratios = []
for d in range(len(comp)):
  ratios.append(comp[d]/nor[d])
compression_metric = mean(ratios)
print(compression_metric)

In [ ]:
if len(df['Spoken'].unique()) -1 == len(source) and len(df['Spoken'].unique()) -1 == len(target) and len(df['Spoken'].unique()) -1 == len(target_compressed):
  with open('source.en','w') as writefile:
          for line in source:
                writefile.writelines(line + '\n')
  with open('target.de','w') as writefile:
          for line in target:
                writefile.writelines(line + '\n')
  with open('target_compressed.de','w') as writefile:
          for line in target_compressed:
                writefile.writelines(line + '\n')
else:
  print('Error')

In [ ]:
!ls -l
!perl scripts/tokenizer.perl -no-escape -l de -threads 16 < target.de > target.tok.de
!perl scripts/tokenizer.perl -no-escape -l de -threads 16 < target_compressed.de > target_compressed.tok.de

In [ ]:
%run scripts/train_test_dev.py --source=source.en --target=target.tok.de 

In [ ]:
!pip install subword-nmt

In [ ]:
!subword-nmt learn-bpe -s 50000 < train.tok.clean.trg > bpe.trg

In [ ]:
!subword-nmt apply-bpe -c bpe.trg < train.tok.clean.trg > train.tok.clean.bpe.trg

In [ ]:
!subword-nmt apply-bpe -c bpe.trg < test.tok.clean.trg > test.tok.clean.bpe.trg

In [ ]:
!subword-nmt apply-bpe -c bpe.trg < dev.tok.clean.trg > dev.tok.clean.bpe.trg

In [ ]:
%run scripts/train_test_dev.py --source=source.en --target=target_compressed.tok.de 

In [ ]:
!subword-nmt apply-bpe -c bpe.trg < train.tok.clean.trg > train_compressed.tok.clean.bpe.trg

In [ ]:
!subword-nmt apply-bpe -c bpe.trg < test.tok.clean.trg > test_compressed.tok.clean.bpe.trg

In [ ]:
!subword-nmt apply-bpe -c bpe.trg < dev.tok.clean.trg > dev_compressed.tok.clean.bpe.trg

In [ ]:
#!wc -l bpe.trg

In [ ]:
#!onmt_build_vocab -config /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/config/config_vocab_bpe.yaml

In [ ]:
#!wc -l /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/data/vocab.trg
#!head /content/drive/MyDrive/$MYCLASSFOLDER/EN-NL/data/vocab.trg

In [ ]:
#%run scripts/train_test_dev.py --source=source.en --target=target_compressed.tok.de 

In [ ]:
def ttr_mattr(x):
  text = """"""
  for i in x:
    text += i 
  lexc = LexicalRichness(text)

  ttr_c = lexc.ttr
  mattr_c = lexc.mattr
  print('TTR =', round(ttr_c, 3), 'MATTR =', round(mattr_c(25), 3))
  return(round(ttr_c, 3), round(mattr_c(25), 3))

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Baseline/test.tok.clean.bpe.trg','r') as r: #DGS corpus normal
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/lstm_baseline_final.txt','r') as r: #Baseline LSTM output
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/trans_baseline_final.txt','r') as r: #Baseline Transformer output
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Pipeline/test_compressed.tok.clean.bpe.trg','r') as r: #DGS corpus compressed
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Pipeline1_lstm_translation (modded)','r') as r: #Pipeline LSTM output 1
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/lstm_pipeline_final.txt','r') as r: #Pipeline LSTM output 2
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation (modded)','r') as r: #Pipeline Transformer output 1
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/trans_pipeline_final.txt','r') as r: #Pipeline Transformer output 2
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Baseline_trans_translation_run2','r') as r: #Baseline Transformer run2
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation_run2','r') as r: #Pipeline1 Transformer run2
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Pipeline2_trans_translation_run2','r') as r: #Pipeline2 Transformer run2
          ttr_mattr(r)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Baseline/train.tok.clean.src','r') as r:
  count = 0
  for i in r:
    if len(i) == 1:
      count += 1
  print(count)

In [ ]:
with open('/content/drive/My Drive/DSS_Thesis_code/Pipeline/train_compressed.tok.clean.bpe.trg','r') as b:
          ttr_mattr(b)

In [ ]:
#@title
#REMOVE EMPTY LINES BY INDEX IN Baseline/train.tok.clean.src
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/train.tok.clean.src', encoding = "utf-8") as z:
  data = z.readlines()
  index_list = []
  for index in range(len(data)):
    if data[index] == '\n':
      index_list.append(index)
for i in reversed(index_list):
  del data[i]    

     

with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/train.tok.clean.src', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY MATCHED INDEX IN Baseline/train.tok.clean.bpe.trg
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/train.tok.clean.bpe.trg', encoding = "utf-8") as z:
  data = z.readlines()
for i in reversed(index_list):
  del data[i]    
  

with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/train.tok.clean.bpe.trg', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY INDEX IN Baseline/test.tok.clean.src
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/test.tok.clean.src', encoding = "utf-8") as z:
  data = z.readlines()
  index_list = []
  for index in range(len(data)):
    if data[index] == '\n':
      index_list.append(index)
for i in reversed(index_list):
  del data[i]    

with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/test.tok.clean.src', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY MATCHED INDEX IN Baseline/test.tok.clean.bpe.trg
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/test.tok.clean.bpe.trg', encoding = "utf-8") as z:
  data = z.readlines()
for i in reversed(index_list):
  del data[i]    
  

with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/test.tok.clean.bpe.trg', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY INDEX IN Baseline/dev.tok.clean.src
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/dev.tok.clean.src', encoding = "utf-8") as z:
  data = z.readlines()
  index_list = []
  for index in range(len(data)):
    if data[index] == '\n':
      index_list.append(index)
for i in reversed(index_list):
  del data[i]    

     

with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/dev.tok.clean.src', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY MATCHED INDEX IN Baseline/train.tok.clean.bpe.trg
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/dev.tok.clean.bpe.trg', encoding = "utf-8") as z:
  data = z.readlines()
for i in reversed(index_list):
  del data[i]    
  

with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline/dev.tok.clean.bpe.trg', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY INDEX IN Baseline/train.tok.clean.src
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/train.tok.clean.src', encoding = "utf-8") as z:
  data = z.readlines()
  index_list = []
  for index in range(len(data)):
    if data[index] == '\n':
      index_list.append(index)
for i in reversed(index_list):
  del data[i]    

     

with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/train.tok.clean.src', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY MATCHED INDEX IN Baseline/train.tok.clean.bpe.trg
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/train_compressed.tok.clean.bpe.trg', encoding = "utf-8") as z:
  data = z.readlines()

for i in reversed(index_list):
  del data[i]    
  

with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/train_compressed.tok.clean.bpe.trg', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY INDEX IN Baseline/test.tok.clean.src
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/test.tok.clean.src', encoding = "utf-8") as z:
  data = z.readlines()
  index_list = []
  for index in range(len(data)):
    if data[index] == '\n':
      index_list.append(index)
for i in reversed(index_list):
  del data[i]    

with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/test.tok.clean.src', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY MATCHED INDEX IN Baseline/test.tok.clean.bpe.trg
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/test_compressed.tok.clean.bpe.trg', encoding = "utf-8") as z:
  data = z.readlines()
for i in reversed(index_list):
  del data[i]    
  

with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/test_compressed.tok.clean.bpe.trg', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
#@title
#REMOVE EMPTY LINES BY INDEX IN Baseline/dev.tok.clean.src
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/dev.tok.clean.src', encoding = "utf-8") as z:
  data = z.readlines()
  index_list = []
  for index in range(len(data)):
    if data[index] == '\n':
      index_list.append(index)
for i in reversed(index_list):
  del data[i]  

with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/dev.tok.clean.src', "w") as file_object:
    for v in data:
      file_object.writelines(v) 

In [ ]:
#@title
#REMOVE EMPTY LINES BY MATCHED INDEX IN Baseline/train.tok.clean.bpe.trg
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/dev_compressed.tok.clean.bpe.trg', encoding = "utf-8") as z:
  data = z.readlines()
for i in reversed(index_list):
  del data[i]    
  

with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline/dev_compressed.tok.clean.bpe.trg', "w") as file_object:
    for v in data:
      file_object.writelines(v)

In [ ]:
!onmt_build_vocab -config /content/drive/MyDrive/DSS_Thesis_code/config_vocab_bpe_baseline.yaml #create vocab based on the baseline data set

In [ ]:
!wc -l /content/drive/MyDrive/DSS_Thesis_code/data/vocab.trg
!head /content/drive/MyDrive/DSS_Thesis_code/data/vocab.trg

In [ ]:
!onmt_train -config /content/drive/MyDrive/DSS_Thesis_code/config_train_lstm_bpe_baseline.yaml

In [ ]:
#Peak validation accuracy at 24k steps 

In [ ]:
!onmt_build_vocab -config /content/drive/MyDrive/DSS_Thesis_code/config_vocab_bpe_pipeline.yaml

In [ ]:
!onmt_train -config /content/drive/MyDrive/DSS_Thesis_code/config_train_lstm_bpe_pipeline.yaml

In [ ]:
!ls

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_lstm_translation', encoding = "utf-8") as z: #REMOVE WHITELINES
  data = z.readlines()
  new = []
  for index in range(4768, len(data) ):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
  for a in new:
    print(a)
  
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_lstm_translation', "w") as o:
  for v in new:
    o.writelines(v + '\n')

    

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
  for a in new:
    print(a)
  
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline_trans_translation', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline_trans_translation', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline_trans_translation_run2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline_trans_translation_run2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation_run2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation_run2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation_run2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_trans_translation_run2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/pipeline2_trans_translation_run2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/pipeline2_trans_translation_run2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/pipeline2_trans_translation_run2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/pipeline2_trans_translation_run2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_lstm_translation_run2_2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline1_lstm_translation_run2_2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline_lstm_translation_run2_2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/Baseline_lstm_translation_run2_2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/trans_pipeline1_run2_final.txt', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/trans_pipeline1_run2_final.txt', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline2_lstm_translation_run2_2', encoding = "utf-8") as z:
  data = z.readlines()
  new = []
  for index in range(len(data)):
    if data[index] == '\n':
      continue
    
    if data[index].split()[0] == 'PRED' and data[index].split()[1] != 'SCORE:':

      new.append(data[index].split()[2:])

  for index in range(len(new)):
    new[index] = ' '.join([str(i) for i in new[index]])
    
with open(r'/content/drive/My Drive/DSS_Thesis_code/Pipeline2_lstm_translation_run2_2', "w") as o:
  for v in new:
    o.writelines(v + '\n')

In [ ]:
!pip install sacrebleu

In [ ]:
import os
cwd = os.getcwd()
cwd

In [ ]:
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline2_lstm_translation > lstm_pipeline_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline2_trans_translation > trans_pipeline_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Baseline_lstm_translation > lstm_baseline_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Baseline_trans_translation > trans_baseline_final.txt

In [ ]:
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline1_lstm_translation > lstm_pipeline1_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline1_trans_translation > trans_pipeline1_final.txt

In [ ]:
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Baseline_trans_translation_run2 > trans_baseline_run2_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline1_trans_translation_run2 > trans_pipeline1_run2_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline2_trans_translation_run2 > trans_pipeline2_run2_final.txt

In [ ]:
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Baseline_lstm_translation_run2_2 > trans_baseline_run2_2_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline1_lstm_translation_run2_2 > trans_pipeline1_run2_2_final.txt
!perl scripts/detokenize.perl -no-escape -l de -threads 16 < Pipeline2_lstm_translation_run2_2 > trans_pipeline2_run2_2_final.txt

In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i trans_baseline_run2_2_final.txt -m bleu ter  -b -w

In [ ]:
!sacrebleu test_compressed.tok.clean.bpe.trg -i trans_pipeline1_run2_2_final.txt -m bleu ter  -b -w

In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i trans_pipeline2_run2_2_final.txt -m bleu ter  -b -w

In [ ]:
!sacrebleu test_compressed.tok.clean.bpe.trg -i trans_pipeline1_run2_final.txt -m bleu ter  -b -w 4 #BLUE MST 1 RUN2 Trans

In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i trans_pipeline2_run2_final.txt -m bleu ter  -b -w 4

In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i trans_baseline_run2_final.txt -m bleu ter  -b -w 4

In [ ]:
!sacrebleu test_compressed.tok.clean.bpe.trg -i lstm_pipeline1_final.txt -m bleu ter  -b -w 4 #BLEU MTS1 lstm

In [ ]:
!sacrebleu test_compressed.tok.clean.bpe.trg -i trans_pipeline1_final.txt -m bleu ter  -b -w 4

In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i lstm_baseline_final.txt -m bleu ter  -b -w 4

In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i lstm_pipeline_final.txt -m bleu ter  -b -w 4


In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i trans_baseline_final.txt -m bleu ter -b -w 4

In [ ]:
!sacrebleu test.tok.clean.bpe.trg -i trans_pipeline_final.txt -m bleu ter  -b -w 4

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = ['Trans_B2', 'Trans_B1', 'LSTM_B', 'Trans_P2', 'LSTM_P', 'Trans_P1']
students = [2.6023,2.4522,1.2025,0.4965,0.4807,0.2236 ]
ax.bar(langs,students)
ax.set_ylabel('BLEU scores')
plt.show()

# Some example data to display


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np

In [ ]:
LSTM_P = np.array([0.409, 0.092, 0.008])
Trans_P1 = np.array([0.409, 0.401, 0.02])
Trans_P2 = np.array([0.409, 0.192, 0.014])
LSTM__B = np.array([0.249, 0.089])
Trans_B = np.array([0.249, 0.228])

In [ ]:
plt.plot(LSTM_P, '--', color='c', label = 'LSTM Pipeline')
plt.plot(Trans_P1, '-.', color='#008000', label = 'Transformer Pipeline 1')
plt.plot(Trans_P2, color='b', label = 'Transformer Pipeline 2')
plt.legend()
plt.ylabel('Type-Token Ratio')
plt.xticks(np.arange(1,3))

In [ ]:
from scipy import stats
import numpy as np

In [ ]:
LSTM_P_bl = np.array([0.8535, 0.4807])
Trans_P1_bl = np.array([2.5663, 0.02236])
Trans_P2_bl = np.array([2.5943, 0.4965])

In [ ]:
LSTM_P_ttr = np.array([0.092, 0.008])
Trans_P1_ttr = np.array([0.401, 0.02])
Trans_P2_ttr = np.array([0.192, 0.014])

In [ ]:
bleu = np.array([0.8535, 0.4807,2.5663, 0.02236,2.5943, 0.4965])
ttr = np.array([0.092, 0.008,0.401, 0.02,0.192, 0.014])
ter = np.array([97.6556, 102.2522,96.8974, 101.2857, 95.5461, 100.8480])
mattr = np.array([0.778, 0.605,0.961, 0.732, 0.74, 0.652])

In [ ]:
stats.spearmanr(bleu, ttr)

In [ ]:
stats.spearmanr(ter, ttr)

In [ ]:
stats.spearmanr(bleu, mattr)

In [ ]:
stats.spearmanr(ter, mattr)